In [1]:
import requests,re,datetime
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
c=pd.read_excel(r'抽奖微博.xlsx')
def rep(x):
    return str(x).replace('-','')[0:8]
c['开奖时间']=c['开奖时间'].apply(rep)
c['发布时间']=c['发布时间'].apply(rep)
def timedelta(x):
    datetime.datetime.now().strftime('%Y%m%d')
    now=datetime.datetime.now().strftime('%Y%m%d')
    out=int(x)-int(now)
    return out
c['还剩几天']=c['开奖时间'].apply(timedelta)

In [3]:
headers = {

'Accept': 'text/html, application/xhtml+xml, application/xml; q=0.9, */*; q=0.8',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language': 'zh-CN',
'Connection':'Keep-Alive',
'Cookie': 'wb_view_log=1536*8641.25; SINAGLOBAL=2056188645548.1892.1534946300336; ALF=1566482313; ULV=1543585752977:3:2:2:1422179051951.1645.1543585752561:1543581912199; SUHB=0ebssnM7ZIopAa; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9WFzd3woboF_FaNwvMU8VmOW; SUB=_2AkMrXb_5f8NxqwJRmPETy2ngaYlzzgvEieKdAU4iJRMxHRl-yT83qlIltRB6AN2RFgY5bnhL9VHqaxV8E-7LgCy7n-KQ; YF-Page-G0=86b4280420ced6d22f1c1e4dc25fe846; _s_tentry=-; Apache=1422179051951.1645.1543585752561',
'Host': 'weibo.com',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134',
}

In [4]:
def update(url):
    try:
        r=requests.get(url,headers=headers)
        html=r.text
        soup=BeautifulSoup(html,'lxml')
        a=soup.get_text()
        res_tr = r'&#xe607;<\\/em><em>(.*?)<\\/em><\\/span><\\/span><\\/span><\\/a>\\n'
        m_tr = re.findall(res_tr, a, re.S | re.M)[0]
        res_tr2 = r'&#xe608;<\\/em><em>(.*?)<\\/em><\\/span><\\/span><\\/span><\\/a>\\n'
        m_tr2 = re.findall(res_tr2, a, re.S | re.M)[0]
        res_tr3 = r'<em class=\\"W_ficon ficon_praised S_txt2\\">ñ<\\/em><em>(.*?)<\\/em><\\/span>'
        m_tr3 = re.findall(res_tr3, a, re.S | re.M)[0]
        return int(m_tr),int(m_tr2),int(m_tr3)
    except IndexError:
        return 0,0,0

In [5]:
for i in range(0,len(c)):
    if c.loc[i,'还剩几天']>=0:
        c.loc[i,'转发数'],c.loc[i,'评论数'],c.loc[i,'点赞数']=update(c.loc[i,'该微博链接'])
c[c['还剩几天']>=0]

,该微博链接,发布时间,开奖时间,时间间隔,中奖人数,是否需要关注,是否需要点赞,是否需要转发,是否需要@好友,奖品名称,奖品类型,折合金额,转发数,评论数,点赞数,uid,粉丝数,参与人数,还剩几天
36,https://weibo.com/5480176273/H6jTo5KUN?type=co...,20181208,20181224,16,1,1,1,0,0,iPad,实物,2000.0,4681,2633,37667,5480176273,672045,33026,2
46,https://weibo.com/2878044234/H60lZ4I8O?type=co...,20181206,20190101,26,5,1,0,1,0,冰糖橙,实物,30.0,8979,557,1130,2878044234,25335,8979,8879
72,https://weibo.com/2526981611/H6sOXbfyO?type=co...,20181209,20181225,16,1,1,0,0,0,手机,现金,10100.0,10056,6209,60155,2526981611,1144976,8474,3


In [6]:
del c['还剩几天']

In [7]:
def getuid(x):
    return x[18:28]
c['uid']=c['该微博链接'].apply(getuid)

In [8]:
import json
import urllib.request
#设置代理IP
proxy_addr="122.241.72.191:808"
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data
def get_userfans(id):
    try:
        url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+str(id)
        data=use_proxy(url,proxy_addr)
        content=json.loads(data).get('data')
        fensi=content.get('userInfo').get('followers_count')
        print(str(id)+':'+str(fensi))
        return fensi
    except AttributeError:
        print(str(id)+'出错了')
        return 0

In [9]:
c['粉丝数']=c['uid'].apply(get_userfans)

3514064555:2221593
3992272124:55633
2827386331:3628425
5289935988:1250402
3485942814:17328
1936656463:8989
2141347175:9683536
5883203913:273372
5581785513:9443504
2140284871:3702621
1787971040:254441
5859973320:2716778
3992272124:55633
1560021134:2136830
3160553390:718
5861184617:608984
1638782947:16993822
3249343090:230216
2150511032:13175618
2269960815:21563
1779593415:5389619
1708159150:561813
3867009760:3407
1623235192:129828
3391520684:1602710
1895300977:2742194
5984978677:2416580
1524748171:1213248
1904228041:7089261
1672384324:8967244
1726918143:10915408
5143897135:532564
5847936687:260593
2075787667:1378703
1176222671:177334
2116541280:1170972
5480176273:674754
5472857989:785578
6345465312:702506
3228717734:1853974
1747206594:229501
5897929902:199128
2793908930:151201
1916401801:6485948
1749139802:762745
5124838719:17129
2878044234:25328
1805250771:2030762
2919177471:31732
6392713773:220849
3249343090:230217
2684526787:651352
2871452950:562643
5289935988:1250402
5047072638:3120

In [10]:
c.to_excel('抽奖微博.xlsx',index=None)

In [11]:
c['参与人数']=0
for i in range(0,len(c)):
    if (c.loc[i,'是否需要转发']==1)&(c.loc[i,'是否需要点赞']==0):
        c.loc[i,'参与人数']=c.loc[i,'转发数']
    if (c.loc[i,'是否需要转发']==0)&(c.loc[i,'是否需要点赞']==1):
        c.loc[i,'参与人数']=c.loc[i,'点赞数']
    if (c.loc[i,'是否需要转发']==1)&(c.loc[i,'是否需要点赞']==1):
        c.loc[i,'参与人数']=min(c.loc[i,'点赞数'],c.loc[i,'转发数'])
    if (c.loc[i,'是否需要转发']==0)&(c.loc[i,'是否需要点赞']==0):
        c.loc[i,'参与人数']=c.loc[i,'转发数']

In [12]:
c

,该微博链接,发布时间,开奖时间,时间间隔,中奖人数,是否需要关注,是否需要点赞,是否需要转发,是否需要@好友,奖品名称,奖品类型,折合金额,转发数,评论数,点赞数,uid,粉丝数,参与人数
0,https://weibo.com/3514064555/H4WxXbCfx?type=re...,20181129,20181130,1,3,1,0,1,0,AI音箱,实物,400.00,932,763,326,3514064555,2221593,932
1,https://weibo.com/3992272124/H4rx33ZDZ?type=co...,20181126,20181127,1,1,1,0,1,0,现金,现金,220.00,1078,347,466,3992272124,55633,1078
2,https://weibo.com/2827386331/H4hiN2KRZ?type=co...,20181125,20181205,10,1,0,1,0,0,iPhone XS Max,实物,10100.00,7678,4651,45454,2827386331,3628425,45454
3,https://weibo.com/5289935988/H4kJq51kd?type=co...,20181125,20181206,11,1,1,0,0,0,魅族手机,实物,1000.00,8879,1747,4833,5289935988,1250402,8879
4,https://weibo.com/3485942814/H4gaXu51y?type=co...,20181125,20181126,1,1,1,0,1,0,现金,现金,50.00,148,55,66,3485942814,17328,148
5,https://weibo.com/1936656463/H41ACoVSm?type=re...,20181123,20181126,3,5,1,0,1,0,朱光潜《谈美》,实物,15.00,965,186,115,1936656463,8989,965
6,https://weibo.com/2141347175/H4bOe1wWi?type=co...,20181124,20181125,1,1,1,1,0,0,现金,现金,520.00,619,1052,9985,2141347175,9683536,9985
7,https://weibo.com/5883203913/H4bRHqDG8?type=co...,20181124,20181201,7,1,1,0,1,0,现金,现金,88.00,342,50,77,5883203913,273372,342
8,https://weibo.com/5581785513/H41x8sFrH?type=co...,20181123,20181124,1,1,1,1,0,0,iPhone XS Max,实物,10100.00,14056,8694,68215,5581785513,9443504,68215
9,https://weibo.com/2140284871/H3SXeminB?type=co...,20181122,20181123,1,1,1,1,0,0,现金,现金,520.00,378,661,6056,2140284871,3702621,6056


In [13]:
c.to_excel('抽奖微博.xlsx',index=None)